In [1]:
# Install recognize-anything as a package
! pip install git+https://github.com/xinyu1205/recognize-anything.git

  Cloning https://github.com/xinyu1205/recognize-anything.git to /tmp/pip-req-build-pomaa_fx
  Running command git clone --filter=blob:none --quiet https://github.com/xinyu1205/recognize-anything.git /tmp/pip-req-build-pomaa_fx
  Resolved https://github.com/xinyu1205/recognize-anything.git to commit 9752f2d60ea617b985598041961a0eea977edd60
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-gbyc5wx9/clip_ed115f5048d34e0aa2649b4de08b7854
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-gbyc5wx9/clip_ed115f5048d34e0aa2649b4de08b7854
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 6.7 MB/s eta 0:00:0000:01
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 26.3 MB

In [2]:
# download tag2text checkpoint
# https://huggingface.co/spaces/xinyu1205/recognize-anything/blob/main/tag2text_swin_14m.pth
import urllib.request
urllib.request.urlretrieve("https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/tag2text_swin_14m.pth", "tag2text_swin_14m.pth")

('tag2text_swin_14m.pth', <http.client.HTTPMessage at 0x7f526024eee0>)

In [4]:
'''
 * The Tag2Text Model
 * Written by Xinyu Huang
'''
from argparse import Namespace
import numpy as np
import random

import torch

from PIL import Image
from ram.models import tag2text
from ram import inference_tag2text as inference
from ram import get_transform


In [7]:
args = Namespace(
description='Tag2Text inferece for tagging and captioning',
image_size=384,
pretrained='tag2text_swin_14m.pth',
image='image classification images/lifestyle/1.jpg',
thre=0.68,
specified_tags='sugar')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = get_transform(image_size=args.image_size)

# delete some tags that may disturb captioning
# 127: "quarter"; 2961: "back", 3351: "two"; 3265: "three"; 3338: "four"; 3355: "five"; 3359: "one"
delete_tag_index = [127,2961, 3351, 3265, 3338, 3355, 3359]

#######load model
model = tag2text(pretrained=args.pretrained,
                image_size=args.image_size,
                vit='swin_b',
                delete_tag_index=delete_tag_index)
model.threshold = args.thre  # threshold for tagging
model.eval()

model = model.to(device)


/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
tag2text_swin_14m.pth
--------------
load checkpoint from tag2text_swin_14m.pth
vit: swin_b


In [26]:
import os
for i in os.listdir('image classification images/Product'):
    print(i)
    args = Namespace(
    description='Tag2Text inferece for tagging and captioning',
    image_size=384,
    pretrained='tag2text_swin_14m.pth',
    image='image classification images/Product/%s'%i,
    thre=0.68,
    # specify custome tags separated by comma
    specified_tags='')

    image = transform(Image.open(args.image)).unsqueeze(0).to(device)

    res = inference(image, model, args.specified_tags)
    print("Model Identified Tags: ", res[0])
    print("User Specified Tags: ", res[1])
    print("Image Caption: ", res[2])

10.jpg


Model Identified Tags:  drink | cup | straw | flavor | product | bottle | beverage | fruit | juice
User Specified Tags:  None
Image Caption:  a group of bottles with straws and cups with fruit in each flavor
17.jpg
Model Identified Tags:  snack | bar | box
User Specified Tags:  None
Image Caption:  a box of energy bars with chocolate and peanut butter flavored bars
40.jpg
Model Identified Tags:  coffee | cup | flavor | box
User Specified Tags:  None
Image Caption:  a cup of coffee on a white background
4.jpg
Model Identified Tags:  drink | brand | vodka | alcohol | bottle | beverage
User Specified Tags:  None
Image Caption:  a bottle of beverage is the most popular drink
28.jpg
Model Identified Tags:  pasta | noodle | food | cheese | box | broccoli
User Specified Tags:  None
Image Caption:  a box of noodles with broccoli and cheese
68.jpg
Model Identified Tags:  perfume | bottle | man
User Specified Tags:  None
Image Caption:  a bottle of the perfume that was launched
76.jpg
Model Iden

KeyboardInterrupt: 